In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import decomposition

In [28]:
# retrieving data from file. 
x_train = np.loadtxt("x_train.txt") 
  
# This loadedArr is a 2D array, therefore 
# we need to convert it to the original 
# array shape.reshaping to get original 
# matrice with original shape. 
x_train = x_train.reshape( 
    x_train.shape[0], x_train.shape[1] // 73, 73) 

In [31]:
pca = decomposition.PCA(10)
x_train_pca = np.empty((x_train.shape[0],100,10))

for idx, i in enumerate(x_train):
    
    x_train_pca[idx] = pca.fit_transform(i[:,5:])

In [35]:
# reshaping the array from 3D 
# matrice to 2D matrice. 
x_train_reshaped = x_train_pca.reshape(x_train_pca.shape[0], -1) 
  
# saving reshaped array to file. 
np.savetxt("x_train_pca.txt", x_train_reshaped) 